In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
import re


In [2]:
#TEST##
import requests
from bs4 import BeautifulSoup

url = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html"
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
response.raise_for_status()

soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find("table")

print(table.find("tr"))

<tr class="xl7039" height="29" style="mso-height-source:userset;height:22.0pt">
<td align="left" class="xl7139" colspan="2" height="29" style="height:22.0pt;
  mso-ignore:colspan;width:213pt" width="284">My paper on equity risk premiums:</td>
<td class="xl7039" style="width:65pt" width="87"></td>
<td class="xl7039" style="width:65pt" width="87"></td>
<td class="xl7432" colspan="8" style="width:520pt" width="696"><a href="https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4398884">https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4398884</a></td>
</tr>


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/ctryprem.html"

try:
    # Télécharger les infos du site
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    response.raise_for_status()

    # Rechercher la table
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'width': '919'})

    # Récupérer toutes les lignes
    trs = table.find_all('tr')

    # Trouver la ligne d'en-tête (celle qui contient 'Country')
    header_row = None
    for tr in trs:
        tds = tr.find_all('td')
        if tds and 'Country' in tds[0].get_text():
            header_row = tr
            break

    # Extraire les en-têtes
    headers = [td.get_text(strip=True) for td in header_row.find_all('td')]

    # Extraire uniquement les lignes avec exactement 6 <td>
    rows = []
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) == 6 and tr != header_row:
            row = [td.get_text(strip=True).replace('\xa0','').replace('\n',' ').strip() for td in tds]
            rows.append(row)

except Exception as e:
    print("Erreur lors du scraping :", e)

# Créer le DataFrame
df_damodaran = pd.DataFrame(rows, columns=headers)


In [4]:
print(header_row)

<tr height="21" style="height:16.0pt">
<td class="xl7921" height="21" style="height:16.0pt;width:140pt" width="187">Country</td>
<td class="xl8017" style="border-left:none;width:105pt" width="140">Adj. Default
              Spread</td>
<td class="xl8017" style="border-left:none;width:104pt" width="139">Equity Risk
              Premium</td>
<td class="xl8017" style="border-left:none;width:121pt" width="161">Country Risk
              Premium</td>
<td class="xl8017" style="border-left:none;width:121pt" width="161">Corporate Tax
              Rate</td>
<td class="xl8118" style="border-left:none;width:98pt" width="131">Moody's rating</td>
</tr>


In [5]:
print (df_damodaran)

                                     Country  \
0                                  Abu Dhabi   
1                                    Albania   
2                                    Algeria   
3    Andorra               (Principality of)   
4                                     Angola   
..                                       ...   
188                                  Vietnam   
189                                    Yemen   
190                                   Zambia   
191                                 Zimbabwe   
192                                            

    Adj. Default\n              Spread Equity Risk\n              Premium  \
0                                0.49%                              4.99%   
1                                3.56%                              9.13%   
2                                2.98%                              8.35%   
3                                1.58%                              6.46%   
4                                6.44%

In [6]:
# --- 2. Nettoyer les colonnes numériques ---
import pandas as pd

# 1. Nettoyer les noms de colonnes
df_damodaran.columns = (
    df_damodaran.columns
    .str.replace('\n', ' ', regex=False)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

#2. Définir les colonnes numériques
numeric_cols = [
    'Adj. Default Spread',
    'Equity Risk Premium',
    'Country Risk Premium',
    'Corporate Tax Rate',
]

# Convertir uniquement ces colonnes en numérique
for col in numeric_cols:
    df_damodaran[col] = (
        df_damodaran[col]
        .astype(str)
        .str.replace('%', '', regex=False)
        .str.replace(',', '', regex=False)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )
    df_damodaran[col] = pd.to_numeric(df_damodaran[col], errors='coerce')

#3.Définir les colonnes texte
df_damodaran['Country'] = df_damodaran['Country'].astype(str).str.strip()

# 4. Retirer les lignes totalement vides
df_damodaran = df_damodaran.dropna(how='all')

print(df_damodaran.head())


                                   Country  Adj. Default Spread  \
0                                Abu Dhabi                 0.49   
1                                  Albania                 3.56   
2                                  Algeria                 2.98   
3  Andorra               (Principality of)                 1.58   
4                                   Angola                 6.44   

   Equity Risk Premium  Country Risk Premium  Corporate Tax Rate  \
0                 4.99                  0.66               15.00   
1                 9.13                  4.80               15.00   
2                 8.35                  4.02               15.00   
3                 6.46                  2.13               18.98   
4                13.01                  8.68               25.00   

  Moody's rating  
0            Aa2  
1            Ba3  
2             NR  
3           Baa1  
4             B3  


In [7]:
print(df_damodaran.head())
print(df_damodaran.columns.tolist())


                                   Country  Adj. Default Spread  \
0                                Abu Dhabi                 0.49   
1                                  Albania                 3.56   
2                                  Algeria                 2.98   
3  Andorra               (Principality of)                 1.58   
4                                   Angola                 6.44   

   Equity Risk Premium  Country Risk Premium  Corporate Tax Rate  \
0                 4.99                  0.66               15.00   
1                 9.13                  4.80               15.00   
2                 8.35                  4.02               15.00   
3                 6.46                  2.13               18.98   
4                13.01                  8.68               25.00   

  Moody's rating  
0            Aa2  
1            Ba3  
2             NR  
3           Baa1  
4             B3  
['Country', 'Adj. Default Spread', 'Equity Risk Premium', 'Country Risk Pr

In [8]:
# --- 3. Exportation du CSV ---
df_damodaran.to_csv("country_risk_premiums.csv", index=False) #index=false empêche les indices de se retrouver dans le df